In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def normalize(data):
    mean = data.mean()
    std = data.std()
    return (data - mean) / std

In [3]:
def data(split=0.7):
    x = pd.read_csv('train.csv', header=None)
    x = normalize(x)
    x = x.fillna(x.mean())
    y = pd.read_csv('train_labels.csv', header=None)
    x['y'] = y
    x = x.sample(frac=1)
    index = int(split * x.shape[0])
    x_train = x[:index]
    x_test = x[index:]
    y_train = x_train['y']
    y_test = x_test['y']
    del x_train['y']
    del x_test['y']
    return x_train.as_matrix(), y_train.as_matrix(), x_test.as_matrix(), y_test.as_matrix()

In [4]:
def train(x, y, x_test, y_test):
    feature_columns = [tf.feature_column.numeric_column("x", shape=[100])]
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=2,
                                            model_dir="./model")
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x},
        y=y,
        num_epochs=None,
        shuffle=True)
    classifier.train(input_fn=train_input_fn, steps=2000)
    
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)
    accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
    print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

In [5]:
x_train, y_train, x_test, y_test = data()
x_train
train(x_train, y_train, x_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_model_dir': './model', '_keep_checkpoint_max': 5, '_tf_random_seed': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./model/model.ckpt.
INFO:tensorflow:loss = 91.4429, step = 1
INFO:tensorflow:global_step/sec: 374.166
INFO:tensorflow:loss = 1.21155, step = 101 (0.267 sec)
INFO:tensorflow:global_step/sec: 516.044
INFO:tensorflow:loss = 0.323613, step = 201 (0.193 sec)
INFO:tensorflow:global_step/sec: 575.685
INFO:tensorflow:loss = 0.623858, step = 301 (0.173 sec)
INFO:tensorflow:global_step/sec: 539.264
INFO:tensorflow:loss = 0.307469, step = 401 (0.186 sec)
INFO:tensorflow:global_step/sec: 539.15
INFO:tensorflow:loss = 0.550075, step = 501 (0.187 sec)
INFO:tensorflow:global_step/sec: 38